<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab6/Arch_Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_6*

---


## 环境准备

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Fri Dec 27 08:27:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 1021, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 1021 (delta 55), reused 48 (delta 8), pack-reused 900 (from 2)
Receiving objects: 100% (1021/1021), 62.14 MiB | 24.61 MiB/s, done.
Resolving deltas: 100% (624/624), done.


In [4]:
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1517, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1517 (delta 4), reused 0 (delta 0), pack-reused 1511 (from 2)
Receiving objects: 100% (1517/1517), 1.23 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [5]:
%cd Comp_Arch

/content/Comp_Arch


In [6]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/lab6
  remotes/origin/main


In [7]:
!git switch lab6

Branch 'lab6' set up to track remote branch 'lab6' from 'origin'.
Switched to a new branch 'lab6'


In [8]:
!cp lab6/run_optimized.c /content/llama2.c/run_optimized.c
!cp lab6/run_models.py /content/llama2.c/run_models.py

In [9]:
%cd /content/llama2.c

/content/llama2.c


In [10]:
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin >> download.log 2>&1
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin >> download.log 2>&1
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin >> download.log 2>&1

---
## make run

In [11]:
!make run

gcc -O3 -o run run.c -lm
gcc -O3 -o runq runq.c -lm


In [12]:
!./run stories15M.bin

Once upon a time, there was a big yellow van. The van was outside the window. It was in the yard. A boy named Tim and a girl named Sue wanted to ride in the yellow van.
Tim and Sue went to the yellow van. They opened the door and put their toys inside. They were happy. They played with their toys all day. They got in the yellow van and made a big mess.
Tim and Sue's mom came out to see what they were doing. She saw the mess they made. She was not mad. She smiled and said, "Let's clean up together." Tim and Sue agreed. They put their toys away and got their toys. They played in the yellow van and had a fun day.
achieved tok/s: 52.076467


In [13]:
!./run stories42M.bin

Once upon a time, there was a big red ball. The ball had a crazy face. It could laugh and play all day. The ball liked to make people happy. One day, the ball met a little girl named Sue. Sue was sad because she had no friends to play with. The ball wanted to help Sue.
The ball rolled over to Sue and said, "Hi! I am a big red ball. I can make people happy. Would you like to play with me?" Sue looked at the ball and smiled. She said, "Yes, I want to play with you!" Sue and the ball played all day. They had so much fun.
The next day, Sue saw a boy named Tom. Tom was sad because he had no friends. Sue went to Tom and said, "Hi! I am Sue. Let's play together!" Tom smiled and they played all day. Soon, Tom had many friends. Tom and Sue were very happy. The moral of the story is: when you help others, you make friends and be happy too.
achieved tok/s: 17.777778


In [14]:
!./run stories110M.bin

Once upon a time, there was a little girl named Lily. She loved to play outside with her friends. One day, she saw a big, wide mailbox on the corner of the street. 
"W^C


### 脚本测试

In [15]:
!python3 run_models.py

Running model: stories15M.bin with CPU
Run 1 for stories15M.bin...
Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1154, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.10/subprocess.py", line 2021, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/llama2.c/run_models.py", line 35, in <module>
    process = subprocess.run(
  File "/usr/lib/python3.10/subprocess.py", line 505, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1165, in communicate
    self._wait(timeout=sigint_timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1953, in _wait
    time.sleep(delay)
Keyboar

---
## run with GPU

In [16]:
!nvcc -O3 -lcublas -o run_gpu run_optimized.c

In [17]:
!./run_gpu stories15M.bin

Once upon a time, there was a little girl named Lily. She had a lot of toys, but she was very impatient. Her mom always told her to wait, but she never listened. One day, Lily went to the park with her mom. She was so excited to play on the swings and slide. 
But then, Lily saw a big, scary dog. She got scared and started to scream. Her mom told her not to worry and that the dog was friendly. Lily felt better and wanted to pet the dog. She reached out her hand and the dog licked it. Lily giggled and forgot about being scared. 
From that day on, Lily learned that sometimes it's okay to be a little scared and wait for things to be safe. She also learned that sometimes it's better to be patient and make time for new things. And that's the moral of the story: always wait for your mom and try new things, even if they seem scary at first.
achieved tok/s: 27.525221


In [18]:
!./run_gpu stories42M.bin

Once upon a time, there was a little girl named Lily. She loved to play outside in the big green yard. One day, she saw a tall structure that her daddy built for her. It was an unusual structure with lots of twists and turns.
Lily went up to the structure and knocked on the door. A kind old man opened it and invited her in. He asked her if she wanted to spin around in circles with him. Lily was so happy and said yes. They spun around and around until Lily got dizzy.
As she got up to leave, the old man said, "You know, spinning is so much fun! Come back and visit me anytime." Lily smiled and ran back to her house to tell her mommy and daddy about the unusual structure she found. From that day on, Lily visited the old man every week to spin around in circles with him.
achieved tok/s: 13.716063


In [19]:
!./run_gpu stories110M.bin

Once upon a time, there was a little girl named Lily. She loved to play with her toys, especially her dolls. One day, she saw a new doll at the store. It was very pretty, but also very expensive. Lily's mommy told her they couldn't buy it because it cost too much money.
Lily was sad, but she understood. She went home and played with her other toys. Suddenly, she heard a strange noise. She turned around and saw a little rat running across the floor. Lily was scared and didn't know what to do. But then she remembered something her mommy had told her. She called out for help.
Her mommy came running and saw the rat. She grabbed a broom and tried to catch it. But the rat was too fast and ran under the bed. Lily's mommy tried to reach for it, but she couldn't. So, she decided to squeeze under the bed to get it. And guess what? The rat was already gone! Lily was so happy and relieved. She hugged her mommy and thanked her for helping her.
achieved tok/s: 6.119712


### 脚本测试

In [20]:
!python3 run_models.py --use-gpu

Running model: stories15M.bin with GPU
Run 1 for stories15M.bin...
Run 1: 27.751793 tok/s
Run 2 for stories15M.bin...
Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1154, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.10/subprocess.py", line 2021, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/llama2.c/run_models.py", line 35, in <module>
    process = subprocess.run(
  File "/usr/lib/python3.10/subprocess.py", line 505, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1165, in communicate
    self._wait(timeout=sigint_timeout)
  File "/usr/lib/python3.10/subprocess.py"